## This code Create columns like AI_Suggestions, AI_Insights, AI_Drawbacks

In [ ]:

#--------------------------------------------------
# ORIGINAL CODE
#--------------------------------------------------


import pandas as pd, requests, re, os, time, json
from math import ceil

# ---------- CONFIG ----------
SERVER_BATCH = "http://127.0.0.1:7860/batch_generate"
HEALTH = "http://127.0.0.1:7860/health"
OUTPUT_CSV = r"E:\AJAY\powerbi_codes\llm_suggestions.csv"
TIMEOUT = 600
BATCH_SIZE = 1
MAX_TOKENS = 32
TEMPERATURE = 0.5
RETRIES = 2
RETRY_WAIT = 2
# Sentiment thresholds you provided
NEG_THRESH = -0.1
POS_THRESH = 0.1
# ----------------------------

# Attempt to import VADER (if not present, we'll fallback)
try:
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    VADER_AVAILABLE = True
except Exception:
    VADER_AVAILABLE = False

# ----------------------------
# helper: safely extract values from a JSON-like string or raw dict (unchanged from prior)
def extract_from_raw(raw_text, key):
    if raw_text is None:
        return ""
    s = str(raw_text).strip()
    try:
        j = json.loads(s)
        if isinstance(j, dict) and key in j:
            return str(j.get(key) or "").strip()
    except Exception:
        pass
    pattern = r'["\']?\b' + re.escape(key) + r'\b["\']?\s*[:=]\s*["\']([^"\']+)["\']'
    m = re.search(pattern, s, flags=re.IGNORECASE)
    if m:
        return m.group(1).strip()
    idx = s.lower().find(key.lower())
    if idx != -1:
        sub = s[idx: idx + 600]
        m2 = re.search(r'["\']([^"\']{1,600})["\']', sub)
        if m2:
            return m2.group(1).strip()
        m3 = re.search(r'\b' + re.escape(key) + r'\b\s*[:=]\s*([^,}]+)', sub, flags=re.IGNORECASE)
        if m3:
            return m3.group(1).strip().strip('"\' ')
    return ""

# helper: aggressively clean candidate string (remove labels, quotes, braces, stray words)
def clean_candidate(val, raw=None, key=None):
    if val is None:
        val = ""
    val = str(val).strip()
    if (not val) and raw:
        out = extract_from_raw(raw, key)
        if out:
            val = out
    if key and re.search(r'\b' + re.escape(key) + r'\b', val, flags=re.IGNORECASE):
        out = extract_from_raw(val, key)
        if out:
            val = out
    val = re.sub(r'^[\{\[\]\}\s,]+', '', val)
    val = re.sub(r'^\s*["\']?\s*' + re.escape(key) + r'\s*["\']?\s*[:=\-]\s*', '', val, flags=re.IGNORECASE)
    val = re.sub(r'["\']?\s*' + re.escape(key) + r'\s*["\']?\s*[:=\-]\s*', '', val, flags=re.IGNORECASE)
    val = val.strip(' \'"{},:')
    val = re.sub(r'^\s*(suggestion|insight|drawback)\s*[:\-–—]\s*', '', val, flags=re.IGNORECASE)
    if ('"' in val or "{" in val) and len(val) < 2000:
        m = re.search(r'["\']([^"\']{2,400})["\']', val)
        if m:
            candidate = m.group(1).strip()
            if candidate.lower() != (key or "").lower():
                val = candidate
    val = re.sub(r'\s+', ' ', val).strip()
    if len(val) > 500:
        val = val[:497] + "..."
    return val

# ----------------------------
df = dataset.copy()

if 'Feedback' not in df.columns:
    # no feedback -> return dataset with chosen columns present but empty
    df['AI_Suggestion'] = ''
    df['AI_insights'] = ''
    df['AI_Drawbacks'] = ''
    df['sentiment_score'] = float('nan')
    df['sentiment_label'] = 'unknown'
    df['Positive_Token_Count'] = 0
    df['Negative_Token_Count'] = 0
    dataset = df
else:
    # reset & stable RowID for resume mapping
    df = df.reset_index(drop=True)
    df['RowID'] = df.index

    # -------- SENTIMENT (VADER) & token counts --------
    # If available, use VADER to compute compound score and token lexicon counts
    if VADER_AVAILABLE:
        analyzer = SentimentIntensityAnalyzer()
        # compute compound score for each feedback
        df['sentiment_score'] = df['Feedback'].astype(str).apply(lambda t: analyzer.polarity_scores(str(t))['compound'])
        # map to label using provided thresholds
        def _label_from_score(x):
            try:
                if pd.isna(x):
                    return 'unknown'
                if x < NEG_THRESH:
                    return 'negative'
                if x > POS_THRESH:
                    return 'positive'
                return 'neutral'
            except Exception:
                return 'unknown'
        df['sentiment_label'] = df['sentiment_score'].apply(_label_from_score)

        # Token-level counts using VADER lexicon (counts only tokens present in lexicon and with +/-
        lex = analyzer.lexicon  # dict[word] -> valence
        # Preprocess lex keys to set for fast membership tests
        lex_pos = {w for w,v in lex.items() if v > 0}
        lex_neg = {w for w,v in lex.items() if v < 0}
        # tokenization helper: split on non-word characters, lower-case
        token_pattern = re.compile(r"[A-Za-z0-9']+")
        def count_tokens(text):
            t = str(text).lower()
            tokens = token_pattern.findall(t)
            pos = 0
            neg = 0
            # count only true lexicon matches
            for tok in tokens:
                if tok in lex_pos:
                    pos += 1
                elif tok in lex_neg:
                    neg += 1
            return pos, neg

        counts = df['Feedback'].astype(str).apply(count_tokens)
        df['Positive_Token_Count'] = counts.apply(lambda x: int(x[0]) if isinstance(x, (list,tuple)) else 0)
        df['Negative_Token_Count'] = counts.apply(lambda x: int(x[1]) if isinstance(x, (list,tuple)) else 0)

    else:
        # VADER not installed: set safe defaults or simple fallback
        df['sentiment_score'] = float('nan')
        df['sentiment_label'] = 'unknown'
        df['Positive_Token_Count'] = 0
        df['Negative_Token_Count'] = 0

    # -------- LLM / AI columns (resumable cache) --------
    # quick health check: if server unreachable, still return with sentiment columns present
    try:
        _ = requests.get(HEALTH, timeout=5)
        server_up = True
    except Exception:
        server_up = False

    if not server_up:
        # merge empty AI columns and return quickly (keep sentiment columns)
        df['AI_Suggestion'] = ''
        df['AI_insights'] = ''
        df['AI_Drawbacks'] = ''
        dataset = df.drop(columns=['RowID'])
    else:
        # existing LLM/resume logic (kept same; uses OUTPUT_CSV for caching)
        def make_prompt(text):
            t = re.sub(r'[\r\n]+',' ', str(text)).strip()[:300]
            return (
                "You are a concise feedback analyst. RETURN ONLY a single-line JSON object EXACTLY like: "
                '{"suggestion":"...","insight":"...","drawback":"..."} . '
                "Each value <= 12 words. No extra commentary. Respond with JSON only.\n\n"
                f"Feedback: \"{t}\""
            )

        existing = None
        if os.path.exists(OUTPUT_CSV):
            try:
                existing = pd.read_csv(OUTPUT_CSV)
                if 'RowID' not in existing.columns:
                    existing = None
            except Exception:
                existing = None

        n = len(df)
        res = pd.DataFrame({'RowID': df['RowID'], 'AI_Suggestion': ['']*n, 'AI_insights': ['']*n, 'AI_Drawbacks': ['']*n})

        if existing is not None:
            existing = existing[existing['RowID'].isin(df['RowID'])]
            existing = existing.set_index('RowID')
            for col in ['AI_Suggestion','AI_insights','AI_Drawbacks']:
                if col in existing.columns:
                    for rid in existing.index:
                        try:
                            res.loc[res['RowID']==rid, col] = str(existing.loc[rid, col] or "")
                        except Exception:
                            pass

        to_process = [int(x) for x in res.loc[res['AI_Suggestion'].fillna('') == '', 'RowID'].tolist()]

        if to_process:
            total = len(to_process)
            batches = ceil(total / BATCH_SIZE)
            for b in range(batches):
                batch_idxs = to_process[b*BATCH_SIZE : (b+1)*BATCH_SIZE]
                if not batch_idxs:
                    continue
                items = []
                for rid in batch_idxs:
                    fb = df.at[rid, 'Feedback']
                    items.append({"prompt": make_prompt(fb), "max_tokens": MAX_TOKENS, "temperature": TEMPERATURE})
                payload = {"items": items}

                body = None
                last_err = None
                for attempt in range(1, RETRIES+1):
                    try:
                        r = requests.post(SERVER_BATCH, json=payload, timeout=TIMEOUT)
                        r.raise_for_status()
                        body = r.json()
                        break
                    except Exception as e:
                        last_err = str(e)
                        time.sleep(RETRY_WAIT)
                if body is None:
                    for rid in batch_idxs:
                        res.loc[res['RowID']==rid, ['AI_Suggestion','AI_insights','AI_Drawbacks']] = ['', '', '']
                    try:
                        res.to_csv(OUTPUT_CSV, index=False)
                    except Exception:
                        pass
                    continue

                results = body.get('results') if isinstance(body, dict) and 'results' in body else body
                if not results:
                    for rid in batch_idxs:
                        res.loc[res['RowID']==rid, ['AI_Suggestion','AI_insights','AI_Drawbacks']] = ['', '', '']
                    try:
                        res.to_csv(OUTPUT_CSV, index=False)
                    except Exception:
                        pass
                    continue

                for j, resp in enumerate(results):
                    rid = batch_idxs[j]
                    raw_txt = ''
                    s_raw = i_raw = d_raw = ''
                    if isinstance(resp, dict):
                        s_raw = resp.get('suggestion','') or ''
                        i_raw = resp.get('insight','') or ''
                        d_raw = resp.get('drawback','') or ''
                        raw_txt = resp.get('raw','') or resp.get('text','') or ''
                    else:
                        raw_txt = str(resp)

                    s_val = clean_candidate(s_raw, raw=raw_txt, key='suggestion')
                    i_val = clean_candidate(i_raw, raw=raw_txt, key='insight')
                    d_val = clean_candidate(d_raw, raw=raw_txt, key='drawback')

                    if not s_val:
                        s_val = extract_from_raw(raw_txt, 'suggestion')
                    if not i_val:
                        i_val = extract_from_raw(raw_txt, 'insight')
                    if not d_val:
                        d_val = extract_from_raw(raw_txt, 'drawback')

                    if (not s_val or not i_val or not d_val) and raw_txt:
                        try:
                            parsed = json.loads(raw_txt)
                            if isinstance(parsed, dict):
                                if not s_val:
                                    s_val = str(parsed.get('suggestion','') or parsed.get('SUGGESTION','') or '')
                                if not i_val:
                                    i_val = str(parsed.get('insight','') or parsed.get('INSIGHT','') or '')
                                if not d_val:
                                    d_val = str(parsed.get('drawback','') or parsed.get('DRAWBACK','') or '')
                        except Exception:
                            pass

                    if not (s_val or i_val or d_val):
                        lines = [ln.strip() for ln in str(raw_txt).splitlines() if ln.strip()]
                        if lines:
                            if not s_val:
                                s_val = lines[0]
                            if len(lines) > 1 and not i_val:
                                i_val = lines[1]
                            if len(lines) > 2 and not d_val:
                                d_val = lines[2]

                    def finalize(x):
                        if x is None:
                            return ''
                        t = str(x).strip()
                        t = re.sub(r'^\s*(suggestion|insight|drawback)\b\s*[:\-–—]\s*', '', t, flags=re.IGNORECASE)
                        t = t.strip(' \'"{},:')
                        t = re.sub(r'\s+', ' ', t).strip()
                        if len(t) > 500:
                            t = t[:497] + '...'
                        return t

                    res.loc[res['RowID']==rid, 'AI_Suggestion'] = finalize(s_val)
                    res.loc[res['RowID']==rid, 'AI_insights'] = finalize(i_val)
                    res.loc[res['RowID']==rid, 'AI_Drawbacks'] = finalize(d_val)

                try:
                    res.to_csv(OUTPUT_CSV, index=False)
                except Exception:
                    pass

        # merge back into df
        res = res.set_index('RowID')
        df['AI_Suggestion'] = df['RowID'].map(lambda x: res.at[x, 'AI_Suggestion'] if x in res.index else '')
        df['AI_insights'] = df['RowID'].map(lambda x: res.at[x, 'AI_insights'] if x in res.index else '')
        df['AI_Drawbacks'] = df['RowID'].map(lambda x: res.at[x, 'AI_Drawbacks'] if x in res.index else '')

        # remove helper RowID and return
        dataset = df.drop(columns=['RowID'])

# End — Power BI receives 'dataset' containing:
# original columns + AI_Suggestion, AI_insights, AI_Drawbacks, sentiment_score, sentiment_label, Positive_Token_Count, Negative_Token_Count


## This below code create the Service, Staff, Cost, Cleanliness, Communication, Management, Facility, Safety, and Experience. and AI columsn also well

In [ ]:
# Power BI 
#columns and return single unified dataset 

import pandas as pd, requests, re, os, time, json
from math import ceil

# ---------- CONFIG ----------
SERVER_BATCH = "http://127.0.0.1:7860/batch_generate"
HEALTH = "http://127.0.0.1:7860/health"
OUTPUT_CSV = r"E:\AJAY\powerbi_codes\llm_suggestions.csv"
TIMEOUT = 600
BATCH_SIZE = 1
MAX_TOKENS = 64
TEMPERATURE = 0.0
RETRIES = 2
RETRY_WAIT = 2
# Sentiment thresholds you provided
NEG_THRESH = -0.1
POS_THRESH = 0.1
# ----------------------------

# Attempt to import VADER (if not present, we'll fallback)
try:
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    VADER_AVAILABLE = True
except Exception:
    VADER_AVAILABLE = False

# ----------------------------
# helper: safely extract values from a JSON-like string or raw dict (unchanged from prior)
def extract_from_raw(raw_text, key):
    if raw_text is None:
        return ""
    s = str(raw_text).strip()
    try:
        j = json.loads(s)
        if isinstance(j, dict) and key in j:
            return str(j.get(key) or "").strip()
    except Exception:
        pass
    pattern = r'["\']?\b' + re.escape(key) + r'\b["\']?\s*[:=]\s*["\']([^"\']+)["\']'
    m = re.search(pattern, s, flags=re.IGNORECASE)
    if m:
        return m.group(1).strip()
    idx = s.lower().find(key.lower())
    if idx != -1:
        sub = s[idx: idx + 600]
        m2 = re.search(r'["\']([^"\']{1,600})["\']', sub)
        if m2:
            return m2.group(1).strip()
        m3 = re.search(r'\b' + re.escape(key) + r'\b\s*[:=]\s*([^,}]+)', sub, flags=re.IGNORECASE)
        if m3:
            return m3.group(1).strip().strip('"\' ')
    return ""

# helper: aggressively clean candidate string (remove labels, quotes, braces, stray words)
def clean_candidate(val, raw=None, key=None):
    if val is None:
        val = ""
    val = str(val).strip()
    if (not val) and raw:
        out = extract_from_raw(raw, key)
        if out:
            val = out
    if key and re.search(r'\b' + re.escape(key) + r'\b', val, flags=re.IGNORECASE):
        out = extract_from_raw(val, key)
        if out:
            val = out
    val = re.sub(r'^[\{\[\]\}\s,]+', '', val)
    val = re.sub(r'^\s*["\']?\s*' + re.escape(key) + r'\s*["\']?\s*[:=\-]\s*', '', val, flags=re.IGNORECASE)
    val = re.sub(r'["\']?\s*' + re.escape(key) + r'\s*["\']?\s*[:=\-]\s*', '', val, flags=re.IGNORECASE)
    val = val.strip(' \'"{},:')
    val = re.sub(r'^\s*(suggestion|insight|drawback)\s*[:\-–—]\s*', '', val, flags=re.IGNORECASE)
    if ('"' in val or "{" in val) and len(val) < 2000:
        m = re.search(r'["\']([^"\']{2,400})["\']', val)
        if m:
            candidate = m.group(1).strip()
            if candidate.lower() != (key or "").lower():
                val = candidate
    val = re.sub(r'\s+', ' ', val).strip()
    if len(val) > 500:
        val = val[:497] + "..."
    return val

# normalization helper for category values
def normalize_cat_value(v):
    if v is None:
        return ''
    s = str(v).strip()
    s_low = s.lower()
    if 'satisf' in s_low:
        return 'Satisfied'
    if 'not satisfied' in s_low or ('not' in s_low and 'satisf' not in s_low and 'no' in s_low):
        return 'Not Satisfied'
    if 'not mentioned' in s_low or s_low in ('', 'na', 'n/a', 'none'):
        return 'Not Mentioned'
    # fallback heuristics
    if s_low in ('satisfied','yes','positive','good','ok','okay'):
        return 'Satisfied'
    if s_low in ('negative','bad','unsatisfied','no','not ok','poor'):
        return 'Not Satisfied'
    return 'Not Mentioned'

# categories list (order matters for rating)
CATEGORIES = ['Service','Staff','Cost','Cleanliness','Communication','Management','Facility','Safety','Experience']

# ----------------------------
df = dataset.copy()

if 'Feedback' not in df.columns:
    # no feedback -> return dataset with chosen columns present but empty
    df['AI_Suggestion'] = ''
    df['AI_insights'] = ''
    df['AI_Drawbacks'] = ''
    df['sentiment_score'] = float('nan')
    df['sentiment_label'] = 'unknown'
    df['Positive_Token_Count'] = 0
    df['Negative_Token_Count'] = 0
    for cat in CATEGORIES:
        df[cat] = 'Not Mentioned'
    df['Overall_Experience_Rating'] = float('nan')
    dataset = df
else:
    # reset & stable RowID for resume mapping
    df = df.reset_index(drop=True)
    df['RowID'] = df.index

    # -------- SENTIMENT (VADER) & token counts --------
    if VADER_AVAILABLE:
        analyzer = SentimentIntensityAnalyzer()
        df['sentiment_score'] = df['Feedback'].astype(str).apply(lambda t: analyzer.polarity_scores(str(t))['compound'])
        def _label_from_score(x):
            try:
                if pd.isna(x):
                    return 'unknown'
                if x < NEG_THRESH:
                    return 'negative'
                if x > POS_THRESH:
                    return 'positive'
                return 'neutral'
            except Exception:
                return 'unknown'
        df['sentiment_label'] = df['sentiment_score'].apply(_label_from_score)

        lex = analyzer.lexicon
        lex_pos = {w for w,v in lex.items() if v > 0}
        lex_neg = {w for w,v in lex.items() if v < 0}
        token_pattern = re.compile(r"[A-Za-z0-9']+")
        def count_tokens(text):
            t = str(text).lower()
            tokens = token_pattern.findall(t)
            pos = 0
            neg = 0
            for tok in tokens:
                if tok in lex_pos:
                    pos += 1
                elif tok in lex_neg:
                    neg += 1
            return pos, neg
        counts = df['Feedback'].astype(str).apply(count_tokens)
        df['Positive_Token_Count'] = counts.apply(lambda x: int(x[0]) if isinstance(x, (list,tuple)) else 0)
        df['Negative_Token_Count'] = counts.apply(lambda x: int(x[1]) if isinstance(x, (list,tuple)) else 0)
    else:
        df['sentiment_score'] = float('nan')
        df['sentiment_label'] = 'unknown'
        df['Positive_Token_Count'] = 0
        df['Negative_Token_Count'] = 0

    # -----------------------
    # Prepare category columns (load cache if exists)
    # -----------------------
    # If OUTPUT_CSV exists and has RowID + category columns, use it to prefill
    existing_cache = None
    if os.path.exists(OUTPUT_CSV):
        try:
            existing_cache = pd.read_csv(OUTPUT_CSV)
        except Exception:
            existing_cache = None

    # initialize category columns empty (so we can mark which need processing)
    for cat in CATEGORIES:
        df[cat] = ''

    # If cache present and contains categories, copy by RowID
    if existing_cache is not None and 'RowID' in existing_cache.columns:
        # map by RowID (cache may have AI columns too)
        try:
            existing_cache = existing_cache.set_index('RowID')
            for idx in df['RowID']:
                if idx in existing_cache.index:
                    for cat in CATEGORIES:
                        if cat in existing_cache.columns:
                            val = existing_cache.at[idx, cat]
                            if pd.isna(val):
                                val = ''
                            df.at[idx, cat] = str(val)
        except Exception:
            pass

    # find rows that need classification (blank category values)
    to_process = [int(x) for x in df.loc[df[CATEGORIES[0]].fillna('') == '', 'RowID'].tolist()]

    # If there are rows and server is reachable, call LLM to classify per row (row-by-row or small batches)
    try:
        _ = requests.get(HEALTH, timeout=5)
        server_up = True
    except Exception:
        server_up = False

    if to_process and server_up:
        # Build strict prompt template for category classification
        def make_cat_prompt(feedback_text):
            # LLM must return JSON object with keys exactly the category names and values exactly one of:
            # "Satisfied", "Not Satisfied", "Not Mentioned"
            t = re.sub(r'[\r\n]+',' ', str(feedback_text)).strip()[:800]
            prompt = (
                "You are a feedback classification assistant. Analyze the single feedback text and RETURN ONLY a JSON object "
                "with exactly these keys (lowercase or title case accepted): "
                f"{json.dumps(CATEGORIES)}. For each key assign one of these EXACT values: \"Satisfied\", \"Not Satisfied\", or \"Not Mentioned\". "
                "Do NOT include any other keys or commentary. Values must be exactly one of the three options. "
                "If the feedback mentions the category positively, use \"Satisfied\"; if it clearly complains, use \"Not Satisfied\"; "
                "if the feedback does not mention the category, use \"Not Mentioned\". "
                "Return JSON only.\n\n"
                f"Feedback: \"{t}\"\n"
            )
            return prompt

        # We'll call the batch endpoint with items list where each item is the prompt; batch size kept small to avoid server 422s
        # Use BATCH_SIZE configured (often 1)
        batches = ceil(len(to_process) / BATCH_SIZE)
        cache_rows = []
        for b in range(batches):
            batch_idxs = to_process[b*BATCH_SIZE : (b+1)*BATCH_SIZE]
            items = []
            for rid in batch_idxs:
                fb = df.at[rid, 'Feedback']
                items.append({"prompt": make_cat_prompt(fb), "max_tokens": MAX_TOKENS, "temperature": TEMPERATURE})
            payload = {"items": items}
            body = None
            last_err = None
            # try batch endpoint
            for attempt in range(1, RETRIES+1):
                try:
                    r = requests.post(SERVER_BATCH, json=payload, timeout=TIMEOUT)
                    r.raise_for_status()
                    try:
                        body = r.json()
                    except Exception:
                        body = r.text
                    break
                except Exception as e:
                    last_err = str(e)
                    time.sleep(RETRY_WAIT)
            if body is None:
                # mark these rows as Not Mentioned to fail-safe (do not overwrite if cache had value)
                for rid in batch_idxs:
                    for cat in CATEGORIES:
                        if not df.at[rid, cat]:
                            df.at[rid, cat] = 'Not Mentioned'
                continue

            # body may contain 'results' list or be a list/dict with response text
            results = body.get('results') if isinstance(body, dict) and 'results' in body else body
            # if results is dict or string, normalize to list
            if results is None:
                results = [body]
            if isinstance(results, dict):
                # maybe server returned single dict for the whole batch
                results = [results]

            # Iterate responses and extract JSON
            for j, resp in enumerate(results):
                rid = batch_idxs[j]
                raw_txt = ''
                if isinstance(resp, dict):
                    # try common fields
                    raw_txt = resp.get('raw') or resp.get('text') or resp.get('output') or json.dumps(resp)
                else:
                    raw_txt = str(resp)
                # try to parse JSON object from raw_txt
                parsed = None
                try:
                    parsed = json.loads(raw_txt)
                except Exception:
                    # try to find first {...} block
                    m = re.search(r'(\{[\s\S]*\})', raw_txt)
                    if m:
                        try:
                            parsed = json.loads(m.group(1))
                        except Exception:
                            # try replace single quotes
                            try:
                                parsed = json.loads(m.group(1).replace("'", '"'))
                            except Exception:
                                parsed = None
                # If parsed is dict, extract category values
                if isinstance(parsed, dict):
                    for cat in CATEGORIES:
                        # look for keys case-insensitively
                        val = ''
                        for k in (cat, cat.lower(), cat.upper()):
                            if k in parsed:
                                val = parsed[k]
                                break
                        # try any matching key ignoring case
                        if val == '':
                            for k in parsed.keys():
                                if isinstance(k, str) and k.lower() == cat.lower():
                                    val = parsed[k]
                                    break
                        val_norm = normalize_cat_value(val)
                        if val_norm:
                            df.at[rid, cat] = val_norm
                else:
                    # fallback heuristics: try to extract "Service": "Satisfied" like patterns via regex
                    for cat in CATEGORIES:
                        v = extract_from_raw(raw_txt, cat)
                        if v:
                            df.at[rid, cat] = normalize_cat_value(v)
                    # if still any empty, attempt to find "Satisfied" / "Not Satisfied" in raw text per cat label
                    for cat in CATEGORIES:
                        if not df.at[rid, cat]:
                            # try patterns like Service: Satisfied
                            pat = r'(' + re.escape(cat) + r')\s*[:\-]\s*(Satisfied|Not Satisfied|Not Mentioned)'
                            m2 = re.search(pat, raw_txt, flags=re.IGNORECASE)
                            if m2:
                                df.at[rid, cat] = normalize_cat_value(m2.group(2))
                # ensure all categories have a value now (fallback to Not Mentioned)
                for cat in CATEGORIES:
                    if not df.at[rid, cat]:
                        df.at[rid, cat] = 'Not Mentioned'
                # store cache row for persistence
                cache_row = {'RowID': rid}
                for cat in CATEGORIES:
                    cache_row[cat] = df.at[rid, cat]
                cache_rows.append(cache_row)

        # Persist category cache into OUTPUT_CSV (merge with existing cache file preserving other columns)
        try:
            if os.path.exists(OUTPUT_CSV):
                try:
                    old = pd.read_csv(OUTPUT_CSV)
                except Exception:
                    old = pd.DataFrame()
            else:
                old = pd.DataFrame()
            # convert cache_rows to DataFrame
            if cache_rows:
                new_cache_df = pd.DataFrame(cache_rows)
                # merge on RowID (overwrite category cols)
                if not old.empty:
                    old = old.set_index('RowID')
                    new_cache_df = new_cache_df.set_index('RowID')
                    for cid in new_cache_df.index:
                        old.loc[cid, new_cache_df.columns] = new_cache_df.loc[cid]
                    merged = old.reset_index()
                else:
                    merged = new_cache_df.reset_index()
                # save merged
                merged.to_csv(OUTPUT_CSV, index=False)
        except Exception:
            pass

    else:
        # If server not up or nothing to process, fill missing categories with Not Mentioned
        for idx, row in df.iterrows():
            for cat in CATEGORIES:
                if not df.at[idx, cat]:
                    df.at[idx, cat] = 'Not Mentioned'

    # -----------------------
    # Compute Overall_Experience_Rating using specified mapping
    # -----------------------
    score_map = {'Satisfied': 1.0, 'Not Satisfied': 0.0, 'Not Mentioned': 0.5}
    def compute_overall_rating_row(r):
        s = 0.0
        count = 0
        for cat in CATEGORIES:
            lab = r.get(cat, 'Not Mentioned')
            s += score_map.get(lab, 0.5)
            count += 1
        if count == 0:
            return float('nan')
        rating = (s / count) * 5.0
        try:
            return round(float(rating), 1)
        except Exception:
            return float('nan')
    df['Overall_Experience_Rating'] = df.apply(compute_overall_rating_row, axis=1)

    # -------- LLM / AI columns (resumable cache) --------
    # quick health check: if server unreachable, still return with sentiment columns present
    try:
        _ = requests.get(HEALTH, timeout=5)
        server_up = True
    except Exception:
        server_up = False

    if not server_up:
        # merge empty AI columns and return quickly (keep sentiment columns)
        df['AI_Suggestion'] = ''
        df['AI_insights'] = ''
        df['AI_Drawbacks'] = ''
        dataset = df.drop(columns=['RowID'])
    else:
        # existing LLM/resume logic (kept same; uses OUTPUT_CSV for caching)
        def make_prompt(text):
            t = re.sub(r'[\r\n]+',' ', str(text)).strip()[:300]
            return (
                "You are a concise feedback analyst. RETURN ONLY a single-line JSON object EXACTLY like: "
                '{"suggestion":"...","insight":"...","drawback":"..."} . '
                "Each value <= 12 words. No extra commentary. Respond with JSON only.\n\n"
                f"Feedback: \"{t}\""
            )

        existing = None
        if os.path.exists(OUTPUT_CSV):
            try:
                existing = pd.read_csv(OUTPUT_CSV)
                if 'RowID' not in existing.columns:
                    existing = None
            except Exception:
                existing = None

        n = len(df)
        res = pd.DataFrame({'RowID': df['RowID'], 'AI_Suggestion': ['']*n, 'AI_insights': ['']*n, 'AI_Drawbacks': ['']*n})

        if existing is not None:
            # Copy any existing AI columns present in cache
            existing = existing[existing['RowID'].isin(df['RowID'])]
            existing = existing.set_index('RowID')
            for col in ['AI_Suggestion','AI_insights','AI_Drawbacks']:
                if col in existing.columns:
                    for rid in existing.index:
                        try:
                            res.loc[res['RowID']==rid, col] = str(existing.loc[rid, col] or "")
                        except Exception:
                            pass

        to_process_ai = [int(x) for x in res.loc[res['AI_Suggestion'].fillna('') == '', 'RowID'].tolist()]

        if to_process_ai:
            total = len(to_process_ai)
            batches = ceil(total / BATCH_SIZE)
            for b in range(batches):
                batch_idxs = to_process_ai[b*BATCH_SIZE : (b+1)*BATCH_SIZE]
                if not batch_idxs:
                    continue
                items = []
                for rid in batch_idxs:
                    fb = df.at[rid, 'Feedback']
                    items.append({"prompt": make_prompt(fb), "max_tokens": MAX_TOKENS, "temperature": TEMPERATURE})
                payload = {"items": items}

                body = None
                last_err = None
                for attempt in range(1, RETRIES+1):
                    try:
                        r = requests.post(SERVER_BATCH, json=payload, timeout=TIMEOUT)
                        r.raise_for_status()
                        body = r.json()
                        break
                    except Exception as e:
                        last_err = str(e)
                        time.sleep(RETRY_WAIT)
                if body is None:
                    for rid in batch_idxs:
                        res.loc[res['RowID']==rid, ['AI_Suggestion','AI_insights','AI_Drawbacks']] = ['', '', '']
                    try:
                        res.to_csv(OUTPUT_CSV, index=False)
                    except Exception:
                        pass
                    continue

                results = body.get('results') if isinstance(body, dict) and 'results' in body else body
                if not results:
                    for rid in batch_idxs:
                        res.loc[res['RowID']==rid, ['AI_Suggestion','AI_insights','AI_Drawbacks']] = ['', '', '']
                    try:
                        res.to_csv(OUTPUT_CSV, index=False)
                    except Exception:
                        pass
                    continue

                for j, resp in enumerate(results):
                    rid = batch_idxs[j]
                    raw_txt = ''
                    s_raw = i_raw = d_raw = ''
                    if isinstance(resp, dict):
                        s_raw = resp.get('suggestion','') or ''
                        i_raw = resp.get('insight','') or ''
                        d_raw = resp.get('drawback','') or ''
                        raw_txt = resp.get('raw','') or resp.get('text','') or ''
                    else:
                        raw_txt = str(resp)

                    s_val = clean_candidate(s_raw, raw=raw_txt, key='suggestion')
                    i_val = clean_candidate(i_raw, raw=raw_txt, key='insight')
                    d_val = clean_candidate(d_raw, raw=raw_txt, key='drawback')

                    if not s_val:
                        s_val = extract_from_raw(raw_txt, 'suggestion')
                    if not i_val:
                        i_val = extract_from_raw(raw_txt, 'insight')
                    if not d_val:
                        d_val = extract_from_raw(raw_txt, 'drawback')

                    if (not s_val or not i_val or not d_val) and raw_txt:
                        try:
                            parsed = json.loads(raw_txt)
                            if isinstance(parsed, dict):
                                if not s_val:
                                    s_val = str(parsed.get('suggestion','') or parsed.get('SUGGESTION','') or '')
                                if not i_val:
                                    i_val = str(parsed.get('insight','') or parsed.get('INSIGHT','') or '')
                                if not d_val:
                                    d_val = str(parsed.get('drawback','') or parsed.get('DRAWBACK','') or '')
                        except Exception:
                            pass

                    if not (s_val or i_val or d_val):
                        lines = [ln.strip() for ln in str(raw_txt).splitlines() if ln.strip()]
                        if lines:
                            if not s_val:
                                s_val = lines[0]
                            if len(lines) > 1 and not i_val:
                                i_val = lines[1]
                            if len(lines) > 2 and not d_val:
                                d_val = lines[2]

                    def finalize(x):
                        if x is None:
                            return ''
                        t = str(x).strip()
                        t = re.sub(r'^\s*(suggestion|insight|drawback)\b\s*[:\-–—]\s*', '', t, flags=re.IGNORECASE)
                        t = t.strip(' \'"{},:')
                        t = re.sub(r'\s+', ' ', t).strip()
                        if len(t) > 500:
                            t = t[:497] + "..."
                        return t

                    res.loc[res['RowID']==rid, 'AI_Suggestion'] = finalize(s_val)
                    res.loc[res['RowID']==rid, 'AI_insights'] = finalize(i_val)
                    res.loc[res['RowID']==rid, 'AI_Drawbacks'] = finalize(d_val)

                try:
                    # merge res into existing cache as well (so categories saved earlier remain)
                    if os.path.exists(OUTPUT_CSV):
                        try:
                            cache_df = pd.read_csv(OUTPUT_CSV)
                        except Exception:
                            cache_df = pd.DataFrame()
                    else:
                        cache_df = pd.DataFrame()
                    # join res rows into cache_df
                    res_up = res.copy()
                    if 'RowID' in cache_df.columns:
                        cache_df = cache_df.set_index('RowID')
                        res_up = res_up.set_index('RowID')
                        for rid in res_up.index:
                            cache_df.loc[rid, res_up.columns] = res_up.loc[rid]
                        merged_cache = cache_df.reset_index()
                    else:
                        merged_cache = res_up.reset_index()
                    merged_cache.to_csv(OUTPUT_CSV, index=False)
                except Exception:
                    pass

        # merge back into df
        res = res.set_index('RowID')
        df['AI_Suggestion'] = df['RowID'].map(lambda x: res.at[x, 'AI_Suggestion'] if x in res.index else '')
        df['AI_insights'] = df['RowID'].map(lambda x: res.at[x, 'AI_insights'] if x in res.index else '')
        df['AI_Drawbacks'] = df['RowID'].map(lambda x: res.at[x, 'AI_Drawbacks'] if x in res.index else '')

        # remove helper RowID and return
        dataset = df.drop(columns=['RowID'])


## This below code create the suggestions column like top 5 from the Ai_Suggestion, AI_insights, AI_Drawbacks

In [ ]:
# Power BI -> Top-5 suggestions (single column 'suggestion')
# Uses only existing columns: AI_Suggestion, AI_insights, AI_Drawbacks as LLM input context.
# Requires local LLM batch endpoint at SERVER_BATCH returning text in body (may include code fences).

import pandas as pd, requests, json, re, os, time, ast
from math import ceil

# ---------- CONFIG ----------
SERVER_BATCH = "http://127.0.0.1:7860/batch_generate"
HEALTH = "http://127.0.0.1:7860/health"
INSIGHTS_AUDIT_CSV = r"E:\AJAY\powerbi_codes\llm_top5_insights.csv"
# OUTPUT_CACHE_JSON removed per request (no cache.json file writing)
TIMEOUT = 300
RETRIES = 2
RETRY_WAIT = 2
BATCH_SIZE = 1
MAX_TOKENS = 256   # reduced token budget per your ask
TEMPERATURE = 0.0
N_SAMPLE_ROWS = 40  # sample up to 40 rows from the three columns to build context
# ----------------------------

def safe_str(x):
    return "" if x is None else str(x)

def build_context_sample(df, n=N_SAMPLE_ROWS):
    parts = []
    cnt = 0
    for _, r in df.iterrows():
        if cnt >= n:
            break
        s = safe_str(r.get('AI_Suggestion','')).replace('\n',' ').strip()
        i = safe_str(r.get('AI_insights','')).replace('\n',' ').strip()
        d = safe_str(r.get('AI_Drawbacks','')).replace('\n',' ').strip()
        entry_parts = []
        if s: entry_parts.append("S:" + (s[:300]))
        if i: entry_parts.append("I:" + (i[:300]))
        if d: entry_parts.append("D:" + (d[:300]))
        if entry_parts:
            parts.append(" | ".join(entry_parts))
            cnt += 1
    if not parts:
        return ""
    joined = " ||| ".join(parts)
    return joined[:2400]

def call_batch(payload):
    try:
        r = requests.post(SERVER_BATCH, json=payload, timeout=TIMEOUT)
        return r
    except Exception:
        return None

def remove_code_fences_and_cleanup(s):
    if not s or not isinstance(s, str):
        return s
    s = re.sub(r'```(?:json)?\s*', '', s, flags=re.IGNORECASE)
    s = re.sub(r'\s*```', '', s)
    s = re.sub(r'~~~(?:json)?\s*', '', s, flags=re.IGNORECASE)
    s = re.sub(r'\s*~~~', '', s)
    s = s.replace('\\n', '\n').replace('\\\"', '"').replace('\\"', '"')
    return s

def try_parse_json_candidates(s):
    if not s or not isinstance(s, str):
        return None
    s0 = s.strip()
    try:
        obj = json.loads(s0)
        if isinstance(obj, list):
            return obj
        if isinstance(obj, dict) and 'results' in obj and isinstance(obj['results'], list):
            return obj['results']
        if isinstance(obj, dict) and any(k.lower().startswith('suggest') for k in obj.keys()):
            return [obj]
    except Exception:
        pass

    cleaned = remove_code_fences_and_cleanup(s0)
    m = re.search(r'(\[[\s\S]*?\])', cleaned)
    if m:
        cand = m.group(1)
        try:
            obj = json.loads(cand)
            if isinstance(obj, list):
                return obj
        except Exception:
            try:
                obj = json.loads(cand.replace("'", '"'))
                if isinstance(obj, list):
                    return obj
            except Exception:
                pass

    objs = re.findall(r'(\{[\s\S]*?\})', cleaned)
    parsed_objs = []
    for o in objs:
        try:
            parsed = json.loads(o)
            parsed_objs.append(parsed)
        except Exception:
            try:
                parsed = json.loads(o.replace("'", '"'))
                parsed_objs.append(parsed)
            except Exception:
                pass
    if parsed_objs:
        return parsed_objs

    try:
        val = ast.literal_eval(cleaned)
        if isinstance(val, list):
            return val
        if isinstance(val, dict) and 'results' in val and isinstance(val['results'], list):
            return val['results']
    except Exception:
        pass

    return None

def extract_text_candidates(body):
    texts = []
    if body is None:
        return texts
    if isinstance(body, dict):
        for key in ('results','text','output','raw','generated_text','generation','choices'):
            if key in body and body[key]:
                texts.append(body[key])
        if 'results' in body and isinstance(body['results'], list):
            for it in body['results']:
                if isinstance(it, dict):
                    for k in ('text','raw','output'):
                        if k in it and it[k]:
                            texts.append(it[k])
                    texts.append(json.dumps(it, ensure_ascii=False))
                else:
                    texts.append(str(it))
        if 'choices' in body and isinstance(body['choices'], list):
            for ch in body['choices']:
                if isinstance(ch, dict):
                    if 'text' in ch and ch['text']:
                        texts.append(ch['text'])
                    elif 'message' in ch and isinstance(ch['message'], dict):
                        cont = ch['message'].get('content')
                        if cont:
                            texts.append(cont)
                    else:
                        texts.append(json.dumps(ch, ensure_ascii=False))
                else:
                    texts.append(str(ch))
        texts.append(json.dumps(body, ensure_ascii=False))
    elif isinstance(body, list):
        texts.append(json.dumps(body, ensure_ascii=False))
        for it in body:
            texts.append(it if isinstance(it, str) else json.dumps(it, ensure_ascii=False))
    else:
        texts.append(str(body))
    out = []
    seen = set()
    for t in texts:
        if t is None:
            continue
        s = t if isinstance(t, str) else json.dumps(t, ensure_ascii=False)
        if s not in seen:
            out.append(s)
            seen.add(s)
    return out

def clean_suggestion_text(s):
    if not s:
        return ''
    t = str(s).strip()
    t = re.sub(r'^[\s"\']+', '', t)
    t = re.sub(r'[\s"\']+$', '', t)
    t = re.sub(r'\bPriority\b.*$', '', t, flags=re.IGNORECASE).strip()
    toks = t.split()
    if len(toks) > 15:
        return " ".join(toks[:15]) + "..."
    return t

# ---------- MAIN ----------
df = dataset.copy()

for c in ('AI_Suggestion','AI_insights','AI_Drawbacks'):
    if c not in df.columns:
        df[c] = ''

# If INSIGHTS_AUDIT_CSV exists and has cached suggestions, use it and skip LLM call
insights_table = pd.DataFrame([{'suggestion':''} for _ in range(5)], columns=['suggestion'])
try:
    if os.path.exists(INSIGHTS_AUDIT_CSV):
        try:
            cached = pd.read_csv(INSIGHTS_AUDIT_CSV)
            if 'suggestion' in cached.columns:
                # check if there are >=1 non-empty suggestions
                non_empty = cached['suggestion'].astype(str).str.strip().replace('', pd.NA).dropna()
                if len(non_empty) >= 1:
                    # load up to 5 rows from cache but coerce NaN/'nan'/None to empty string
                    out_rows = []
                    for i in range(5):
                        if i < len(cached):
                            raw_val = cached.iloc[i].get('suggestion','')
                            # coerce NaN/None/'nan' to empty string
                            if pd.isna(raw_val) or str(raw_val).strip().lower() in ('nan','none','na','n/a',''):
                                val = ''
                            else:
                                val = str(raw_val).strip()
                            out_rows.append({'suggestion': val})
                        else:
                            out_rows.append({'suggestion': ''})
                    insights_table = pd.DataFrame(out_rows, columns=['suggestion'])
                    dataset = df
                else:
                    pass
        except Exception:
            pass
except Exception:
    pass

# If insights_table already loaded from cache (non-empty), we can skip heavy LLM call:
if not (insights_table['suggestion'].astype(str).str.strip().replace('', pd.NA).dropna().empty):
    # ensure no 'nan' strings remain
    insights_table['suggestion'] = insights_table['suggestion'].apply(lambda x: '' if pd.isna(x) or str(x).strip().lower() in ('nan','none','na','n/a','') else str(x).strip())
    try:
        insights_table.to_csv(INSIGHTS_AUDIT_CSV, index=False, encoding='utf-8')
    except Exception:
        pass
    dataset = df
    insights_table = insights_table.reset_index(drop=True)
else:
    # Proceed to call LLM (same logic as before)
    context_snip = build_context_sample(df)

    PROMPT = (
        "You are an operations improvement expert. Read the supplied text context (three short AI columns) "
        "and produce EXACTLY 5 distinct, concise, actionable suggestions. RETURN ONLY a JSON array of 5 objects; "
        "each object MUST have precisely one key: \"suggestion\" (lowercase). "
        "Each suggestion should be actionable and approximately 10-15 words. "
        "Do NOT include reasons, priorities, numbering, or any commentary. Maintain strictly valid JSON array only.\n\n"
        f"Context (AI_Suggestion / AI_insights / AI_Drawbacks samples): {context_snip}\n\nReturn JSON array now."
    )

    server_up = False
    try:
        h = requests.get(HEALTH, timeout=5)
        server_up = (h.status_code == 200)
    except Exception:
        server_up = False

    if server_up:
        payload = {"items":[{"prompt":PROMPT, "max_tokens": MAX_TOKENS, "temperature": TEMPERATURE}]}
        resp_body = None
        try:
            r = requests.post(SERVER_BATCH, json=payload, timeout=TIMEOUT)
            if r is not None and r.status_code == 200:
                try:
                    resp_body = r.json()
                except Exception:
                    resp_body = r.text
            else:
                resp_body = None
        except Exception:
            resp_body = None

        if resp_body is not None:
            candidates = extract_text_candidates(resp_body)
            parsed = None
            for c in candidates:
                if not c:
                    continue
                cleaned = remove_code_fences_and_cleanup(c)
                parsed_candidate = try_parse_json_candidates(cleaned)
                if parsed_candidate:
                    parsed = parsed_candidate
                    break
                cleaned2 = re.sub(r'\\{2,}', '\\', cleaned)
                parsed_candidate = try_parse_json_candidates(cleaned2)
                if parsed_candidate:
                    parsed = parsed_candidate
                    break

            suggestions = []
            if isinstance(parsed, list) and parsed:
                for el in parsed:
                    if isinstance(el, dict) and 'suggestion' in el:
                        txt = clean_suggestion_text(el.get('suggestion',''))
                        suggestions.append(txt)
                    elif isinstance(el, str):
                        suggestions.append(clean_suggestion_text(el))
                    elif isinstance(el, dict):
                        for v in el.values():
                            if isinstance(v, str) and v.strip():
                                suggestions.append(clean_suggestion_text(v))
                                break
                    if len(suggestions) >= 5:
                        break

            final = []
            seen = set()
            for s in suggestions:
                if s and s not in seen:
                    final.append({'suggestion': s})
                    seen.add(s)
                if len(final) >= 5:
                    break

            while len(final) < 5:
                final.append({'suggestion': ''})

            if final:
                insights_table = pd.DataFrame(final, columns=['suggestion'])

    # persist for audit
    try:
        # ensure no NaN strings before saving
        insights_table['suggestion'] = insights_table['suggestion'].apply(lambda x: '' if pd.isna(x) or str(x).strip().lower() in ('nan','none','na','n/a','') else str(x).strip())
        insights_table.to_csv(INSIGHTS_AUDIT_CSV, index=False, encoding='utf-8')
    except Exception:
        pass

    dataset = df
    insights_table = insights_table.reset_index(drop=True)

# Final outputs (Power BI will pick these variables)
dataset = dataset
insights_table = insights_table


## SCRIPT A — Referrals_Clean (cleaned fact table + derived columns)

In [ ]:
import pandas as pd
import numpy as np
import hashlib

# If your source dates are dd/mm/yyyy set DAYFIRST = True
DAYFIRST = False

# Power Query supplies the table as 'dataset'
df = dataset.copy()

# 1) Parse date columns
date_cols = [
    "Received Date","First Administration Date","Last Administration Date in Period",
    "Service Provided Date","Status Start Date","Status End Date",
    "Therapy Discontinuation Date","Date Archived"
]
for c in date_cols:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors='coerce', dayfirst=DAYFIRST)

# 2) Trim/normalize text columns and fill simple nulls with "Unknown"
text_cols = ["Therapy Category","KPI Frequency","Drug Name","Performance Metric",
             "Current Patient Status","Reason for Status","Main Reason","Insurance / Payer"]
for c in text_cols:
    if c in df.columns:
        df[c] = df[c].astype(str).str.strip().replace({"nan": pd.NA, "None": pd.NA})
        df[c] = df[c].fillna("Unknown")

# 3) Fill ONLY blank Main Reason with specific text
if "Main Reason" in df.columns:
    df["Main Reason"] = df["Main Reason"].replace([None, np.nan, "", " ", "nan"], np.nan)
    df.loc[df["Main Reason"].isna(), "Main Reason"] = "Active / Ongoing Therapy (No Reason Recorded)"

# 4) Fill ONLY blank Date Archived with Therapy Discontinuation Date + 10 days
if ("Date Archived" in df.columns) and ("Therapy Discontinuation Date" in df.columns):
    mask_fill = df["Date Archived"].isna() & df["Therapy Discontinuation Date"].notna()
    df.loc[mask_fill, "Date Archived"] = df.loc[mask_fill, "Therapy Discontinuation Date"] + pd.Timedelta(days=10)
    # keep Date Archived as datetime (do not convert to string here so Calendar can use it)
    df["Date Archived"] = pd.to_datetime(df["Date Archived"], errors='coerce')

# 5) Drop PII (First/Last Name) and anonymize Patient ID
for c in ["First Name","Last Name"]:
    if c in df.columns:
        df = df.drop(columns=[c])
if "Patient ID" in df.columns:
    def anonymize(x):
        try:
            return hashlib.sha1(str(x).encode('utf-8')).hexdigest()[:12]
        except:
            return pd.NA
    df["AnonymizedPatientID"] = df["Patient ID"].apply(anonymize)

# 6) Derived numeric features (durations & flags)
if set(["First Administration Date","Last Administration Date in Period"]).issubset(df.columns):
    df["Therapy_Duration_Days"] = (df["Last Administration Date in Period"] - df["First Administration Date"]).dt.days

if set(["Received Date","First Administration Date"]).issubset(df.columns):
    df["Time_to_First_Admin_Days"] = (df["First Administration Date"] - df["Received Date"]).dt.days

if set(["Status Start Date","Status End Date"]).issubset(df.columns):
    df["Status_Duration_Days"] = (df["Status End Date"] - df["Status Start Date"]).dt.days

if set(["First Administration Date","Therapy Discontinuation Date"]).issubset(df.columns):
    df["Days_To_Discontinuation"] = (df["Therapy Discontinuation Date"] - df["First Administration Date"]).dt.days

df["Is_Discontinued"] = (df["Therapy Discontinuation Date"].notna()).astype(int) if "Therapy Discontinuation Date" in df.columns else 0

# 7) KPI Frequency -> KPI_Days mapping (normalize)
kpi_map = {"Daily":1,"24 hrs":1,"Weekly":7,"Monthly":30,"Quarterly":90,"Annual":365,"On Hold/Seasonal":np.nan,"On Hold":np.nan,"Unknown":np.nan}
if "KPI Frequency" in df.columns:
    df["KPI_Freq_Normalized"] = df["KPI Frequency"].str.strip().replace({
        r"(?i)daily":"Daily", r"(?i)24[^0-9]*hrs?":"24 hrs",
        r"(?i)weekly":"Weekly", r"(?i)monthly":"Monthly",
        r"(?i)quarterly":"Quarterly", r"(?i)annual":"Annual"
    }, regex=True)
    df["KPI_Days"] = df["KPI_Freq_Normalized"].map(kpi_map)

# SLA breach flag
if set(["Time_to_First_Admin_Days","KPI_Days"]).issubset(df.columns):
    df["SLA_Breach_Flag"] = np.where((~df["Time_to_First_Admin_Days"].isna()) & (~df["KPI_Days"].isna()) & (df["Time_to_First_Admin_Days"] > df["KPI_Days"]),1,0)
else:
    df["SLA_Breach_Flag"] = 0

# 8) Therapy duration bucket
bins = [-1,7,30,90,10**9]
labels = ["0-7 days","8-30 days","31-90 days","90+ days"]
if "Therapy_Duration_Days" in df.columns:
    df["Therapy_Duration_Bucket"] = pd.cut(df["Therapy_Duration_Days"].fillna(-1), bins=bins, labels=labels).astype(str)
else:
    df["Therapy_Duration_Bucket"] = np.nan

# 9) QA flag for status dates reversed
if set(["Status Start Date","Status End Date"]).issubset(df.columns):
    df["StatusDate_Error"] = np.where((~df["Status Start Date"].isna()) & (~df["Status End Date"].isna()) & (df["Status End Date"] < df["Status Start Date"]), "EndBeforeStart","")

# 10) Reorder columns (important ones first)
front = ["AnonymizedPatientID","Patient ID","Therapy Category","Drug Name","KPI Frequency","KPI_Freq_Normalized","KPI_Days",
         "Performance Metric","Received Date","First Administration Date","Last Administration Date in Period",
         "Service Provided Date","Current Patient Status","Status Start Date","Status End Date","Status_Duration_Days",
         "Reason for Status","Therapy Discontinuation Date","Days_To_Discontinuation","Is_Discontinued",
         "Main Reason","Insurance / Payer","Date Archived","Therapy_Duration_Days","Therapy_Duration_Bucket",
         "Time_to_First_Admin_Days","SLA_Breach_Flag","StatusDate_Error"]
available = [c for c in front if c in df.columns]
others = [c for c in df.columns if c not in available]
df = df[available + others]

# Return cleaned dataframe back to Power Query
df.reset_index(drop=True, inplace=True)
df


## SCRIPT B — Calendar_DateTable (daily calendar)

In [ ]:
import pandas as pd
import numpy as np

# dataset is Referrals_Clean
df = dataset.copy()

# date columns to inspect (only dates present)
date_cols = [
    "Received Date","First Administration Date","Last Administration Date in Period",
    "Service Provided Date","Status Start Date","Status End Date",
    "Therapy Discontinuation Date","Date Archived"
]
date_cols = [c for c in date_cols if c in df.columns]

# collect non-null date values (floor to date)
dates = []
for c in date_cols:
    try:
        dates += df[c].dropna().dt.floor('D').tolist()
    except:
        # fallback: coerce to datetime then take date
        dates += pd.to_datetime(df[c], errors='coerce').dropna().dt.floor('D').tolist()

# fallback if no dates found
if len(dates) == 0:
    mindate = pd.Timestamp.today().floor('D') - pd.Timedelta(days=365)
    maxdate = pd.Timestamp.today().floor('D') + pd.Timedelta(days=365)
else:
    mindate = min(dates)
    maxdate = max(dates)

# build calendar (daily)
cal = pd.DataFrame({"Date": pd.date_range(start=mindate, end=maxdate, freq='D')})
cal["Year"] = cal["Date"].dt.year
cal["Quarter"] = cal["Date"].dt.quarter
cal["MonthNo"] = cal["Date"].dt.month
cal["MonthName"] = cal["Date"].dt.strftime("%B")
cal["MonthShort"] = cal["Date"].dt.strftime("%b")
cal["YearMonth"] = cal["Date"].dt.strftime("%Y-%m")
cal["WeekNum"] = cal["Date"].dt.isocalendar().week
cal["WeekDay"] = cal["Date"].dt.weekday + 1  # Mon=1..Sun=7
cal["Day"] = cal["Date"].dt.day
cal["DayName"] = cal["Date"].dt.strftime("%A")
cal["IsWeekend"] = np.where(cal["WeekDay"] > 5, 1, 0)
cal["WeekEnding"] = cal["Date"] + pd.to_timedelta(7 - cal["WeekDay"], unit='D')
cal["QuarterNumber"] = "Q" + cal["Quarter"].astype(str)

# Return calendar to Power Query
cal.reset_index(drop=True, inplace=True)
cal


## Step-by-Step to Create and Save the “Image Table”

In [ ]:
import pandas as pd
import numpy as np

# Copy from Power BI input
df = dataset.copy()

# Keep only Insurance / Payer
df = df[['Insurance / Payer']].dropna()
df['Insurance / Payer'] = df['Insurance / Payer'].astype(str).str.strip()

# Drop duplicates
df = df.drop_duplicates(subset=['Insurance / Payer']).reset_index(drop=True)

# Create Image ID (factorized)
df["Image ID"] = pd.factorize(df["Insurance / Payer"])[0] + 1

# Add empty Image column
df["Image"] = ""

# Reorder columns
df = df[["Image ID", "Insurance / Payer", "Image"]]

# Return the table to Power BI
df


## Storing Large Images In Power BI Datasets

In [ ]:
let
  //Get list of files in folder 
  Source = Folder.Files("C:\Users\Chris\Documents\PQ Pics"),
  //Remove unnecessary columns 
  RemoveOtherColumns = Table.SelectColumns(Source, {"Content", "Name"}),
  //Creates Splitter function 
  SplitTextFunction = Splitter.SplitTextByRepeatedLengths(30000),
  //Converts table of files to list 
  ListInput = Table.ToRows(RemoveOtherColumns),
  //Function to convert binary of photo to multiple 
  //text values 
  ConvertOneFile = (InputRow as list) =>
    let
      BinaryIn    = InputRow{0},
      FileName    = InputRow{1},
      BinaryText  = Binary.ToText(BinaryIn, BinaryEncoding.Base64),
      SplitUpText = SplitTextFunction(BinaryText),
      AddFileName = List.Transform(SplitUpText, each {FileName, _})
    in
      AddFileName,
  //Loops over all photos and calls the above function 
  ConvertAllFiles = List.Transform(ListInput, each ConvertOneFile(_)),
  //Combines lists together 
  CombineLists = List.Combine(ConvertAllFiles),
  //Converts results to table 
  ToTable = #table(type table [Name = text, Pic = text], CombineLists),
  //Adds index column to output table 
  AddIndexColumn = Table.AddIndexColumn(ToTable, "Index", 0, 1)
in
  AddIndexColumn

In [ ]:
Display Image = 
IF( HASONEVALUE('PQ Pics'[Name]), 
"data:image/jpeg;base64, " & 
CONCATENATEX( 'PQ Pics', 'PQ Pics'[Pic], , 'PQ Pics'[Index], ASC) )